In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import plotly.express as px
from scipy import stats
import optuna
import pickle

import warnings
warnings.filterwarnings('ignore')

c:\IDE\Skillfactory\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

In [30]:
df = pd.read_pickle('data/df_for_model_3.pickle')
df.head(3)

,status,propertyType,street,baths,fireplace,sqft,beds,stories,target,private_pool,...,cooling,lot_size,school_rate_mean,school_distance_mean,has_parking,garage,parking_spaces,log_target,states_shoted,cities_shorted
1,for sale,single,avenue,3,0,1947.0,3,2,310000.0,0,...,0,5828.0,2.666667,1.326667,0,0,0,12.644331,WA,others
2,for sale,single,road,2,1,3000.0,3,1,2895000.0,1,...,1,8626.0,6.666667,1.960000,1,1,0,14.878496,CA,los angeles
3,for sale,single,avenue,8,1,6457.0,5,3,2395000.0,0,...,1,8220.0,9.250000,0.752500,1,1,0,14.688894,TX,dallas


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348298 entries, 1 to 376608
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                348298 non-null  object 
 1   propertyType          348298 non-null  object 
 2   street                348298 non-null  object 
 3   baths                 348298 non-null  int32  
 4   fireplace             348298 non-null  int32  
 5   sqft                  348298 non-null  float64
 6   beds                  348298 non-null  int32  
 7   stories               348298 non-null  int32  
 8   target                348298 non-null  float64
 9   private_pool          348298 non-null  int64  
 10  year_built            348298 non-null  int32  
 11  heating               348298 non-null  object 
 12  cooling               348298 non-null  int32  
 13  lot_size              348298 non-null  float64
 14  school_rate_mean      348298 non-null  float64
 15  schoo

In [4]:
df.select_dtypes(include=['object'])

,status,propertyType,street,heating,states_shoted,cities_shorted
1,for sale,single,avenue,no_heat,WA,others
2,for sale,single,road,forced_air,CA,los angeles
3,for sale,single,avenue,forced_air,TX,dallas
5,for sale,townhouse,street,forced_air,PA,philadelphia
6,active,other,court,electric,FL,poinciana
...,...,...,...,...,...,...
376604,no_data,single,road,other,FL,miami
376605,for sale,flat,drive,other,IL,chicago
376606,for sale,single,boulevard,other,NY,jamaica
376607,no_data,other,street,no_heat,TX,houston


In [5]:
df.select_dtypes(exclude=['object'])

,baths,fireplace,sqft,beds,stories,target,private_pool,year_built,cooling,lot_size,school_rate_mean,school_distance_mean,has_parking,garage,parking_spaces,log_target
1,3,0,1947.000000,3,2,3.100000e+05,0,2019,0,5828.000000,2.666667,1.326667,0,0,0,12.644331
2,2,1,3000.000000,3,1,2.895000e+06,1,1961,1,8626.000000,6.666667,1.960000,1,1,0,14.878496
3,8,1,6457.000000,5,3,2.395000e+06,0,2006,1,8220.000000,9.250000,0.752500,1,1,0,14.688894
5,1,0,897.000000,2,2,2.090000e+05,0,1920,1,680.000000,1.395706,0.443466,0,0,0,12.250094
6,1,0,1507.000000,1,1,1.815000e+05,0,2006,1,4996.000000,1.750000,3.825000,0,0,0,12.109016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376604,6,0,4017.000000,5,1,1.249000e+06,1,1990,1,8500.000000,7.500000,16.600000,1,0,2,14.037855
376605,3,0,2000.000000,3,9,6.749990e+05,0,1924,0,2120.714286,4.333333,4.143333,0,0,0,13.422468
376606,3,0,1152.000000,3,2,5.280000e+05,0,1950,0,1600.000000,4.500000,0.605000,1,0,2,13.176853
376607,1,0,1985.524752,8,1,2.135520e+05,0,1957,0,6859.959538,0.000000,1.233333,0,0,0,12.271640


In [6]:
df.isna().sum().sum()

0

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348298 entries, 1 to 376608
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                348298 non-null  object 
 1   propertyType          348298 non-null  object 
 2   street                348298 non-null  object 
 3   baths                 348298 non-null  int32  
 4   fireplace             348298 non-null  int32  
 5   sqft                  348298 non-null  float64
 6   beds                  348298 non-null  int32  
 7   stories               348298 non-null  int32  
 8   target                348298 non-null  float64
 9   private_pool          348298 non-null  int64  
 10  year_built            348298 non-null  int32  
 11  heating               348298 non-null  object 
 12  cooling               348298 non-null  int32  
 13  lot_size              348298 non-null  float64
 14  school_rate_mean      348298 non-null  float64
 15  schoo

In [5]:

col_obj = ['status','propertyType','street','heating','states_shoted','cities_shorted']

In [34]:
dft = df.copy()

In [26]:
# num_col = dft.select_dtypes(exclude=['object']).columns.to_list()

In [10]:
num_col = ['baths',
 'fireplace',
 'sqft',
 'beds',
 'stories',
 'private_pool',
 'year_built',
 'cooling',
 'lot_size',
 'school_rate_mean',
 'school_distance_mean',
 'has_parking',
 'garage',
 'parking_spaces']

In [37]:
str_col = dft.select_dtypes(include=['object']).columns.to_list()

In [24]:
str_col

['status',
 'propertyType',
 'street',
 'heating',
 'states_shoted',
 'cities_shorted']

In [35]:
dft.shape

(348298, 22)

In [38]:
dft = pd.concat([dft, pd.get_dummies(dft[str_col])], axis=1)

In [40]:
dft.shape

(348298, 201)

In [41]:
# dft = pd.concat([dft, pd.get_dummies(dft[str_col])], axis=1)
dft = dft.drop(['status','propertyType','street','heating','states_shoted','cities_shorted'], axis=1)

In [42]:
dft.shape

(348298, 195)

In [44]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348298 entries, 1 to 376608
Columns: 195 entries, baths to cities_shorted_winston salem
dtypes: bool(179), float64(6), int32(7), int64(3)
memory usage: 95.3 MB


In [17]:
X = dft.drop(['target'], axis=1)
y = dft['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
X = dft.drop(['target','log_target'], axis=1)
y = dft['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(data=scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=scaler.transform(X_test), columns=X_test.columns)

In [26]:
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [47]:
linrg = Ridge(random_state=42, alpha=0.5)
linrg.fit(X_train_scaled, y_train)
y_train_pred = linrg.predict(X_train_scaled)
y_test_pred  = linrg.predict(X_test_scaled)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))

Train RMSE: 1764888.9775799133
Test RMSE: 2035946.164579697


In [ ]:
# ваш код здесь
lin_regr = LinearRegression()
lin_regr.fit(X_train, y_train_log)
y_pred_train_log = lin_regr.predict(X_train)
y_pred_valid_log = lin_regr.predict(X_test)

print('RMSLE on the train sample: {:.2f}'.format(
    np.sqrt(metrics.mean_squared_error(y_train_log, y_pred_train_log))))
print('RMSLE on the validation sample: {:.2f}'.format(
    np.sqrt(metrics.mean_squared_error(y_test_log, y_pred_valid_log))))

In [37]:
param_grid = [
              {'alpha': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5] , # тип регуляризации
              'solver': ['lsqr', 'sag']} # алгоритм оптимизации
]

grid_search_1 = RandomizedSearchCV(
    estimator=Ridge(random_state=42), 
    param_distributions=param_grid, 
    cv=5, 
    n_jobs = 5,
    n_iter = 10
)  
%time grid_search_1.fit(X_train_scaled, y_train) 
# print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.mean_squared_error(X_test_scaled, y_test)))
# y_test_pred = grid_search_1.predict(X_test_scaled)
# print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
# print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))
y_train_pred = grid_search_1.predict(X_train_scaled)
y_test_pred  = grid_search_1.predict(X_test_scaled)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))

CPU times: total: 16.1 s
Wall time: 2min 34s
Train RMSE: 409490.3318233742
Test RMSE: 408169.2349218025


In [38]:
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

Наилучшие значения гиперпараметров: {'solver': 'sag', 'alpha': 0.3}


In [ ]:
table_dict = dict()
table_columns = ['Train', 'Test']

In [ ]:
table_dict['Linear regression (r12n)'] = [
    mean_squared_error(y_train, y_train_pred, squared=False),
    mean_squared_error(y_test, y_test_pred, squared=False)
]

In [38]:
dectree = DecisionTreeRegressor(random_state=42, criterion='squared_error')

parameters = {
    'splitter' : ['best', 'random'],
    'max_depth' : [4, 5, 6],
    'min_samples_split' : [4, 6, 8]
}

grid_search_2 = RandomizedSearchCV(
    estimator = dectree,
    param_distributions = parameters,
    cv = 5,
    n_jobs=5,
    n_iter=20
)

%time grid_search_2.fit(X_train_scaled, y_train) 
# grid_dt.fit(X_train_scaled, y_train)
# dt_best_params = grid_dt.best_params_
# print('Оптимальные параметры:', grid_dt.best_params_)

y_train_pred = grid_search_2.predict(X_train_scaled)
y_test_pred  = grid_search_2.predict(X_test_scaled)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))


c:\IDE\Skillfactory\.venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: total: 4.05 s
Wall time: 1min 23s
Train RMSE: 420212.9338945511
Test RMSE: 423492.5002186049


In [40]:
print('Оптимальные параметры:', grid_search_2.best_params_)

Оптимальные параметры: {'splitter': 'best', 'min_samples_split': 4, 'max_depth': 6}


In [48]:
best_params = {'splitter': 'best', 'min_samples_split': 4, 'max_depth': 6}

In [24]:
xgb = XGBRegressor(seed=42)
xgb.fit(X_train_scaled, y_train)
y_train_pred = xgb.predict(X_train_scaled)
y_test_pred  = xgb.predict(X_test_scaled)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))

Train RMSE: 606269.3543159844
Test RMSE: 1193810.4011255277


In [52]:
xgb = XGBRegressor(**best_params, seed=42)
xgb.fit(X_train_scaled, y_train)
y_train_pred = xgb.predict(X_train_scaled)
y_test_pred  = xgb.predict(X_test_scaled)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))

[20:08:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "min_samples_split", "splitter" } are not used.

Train RMSE: 606269.3543159844
Test RMSE: 1193810.4011255277


In [41]:
print('Train RMSE:', r2_score(y_train, y_train_pred))
print('Test RMSE:', r2_score(y_test, y_test_pred))

Train RMSE: 0.7391167027160221
Test RMSE: 0.7024977663747415


In [51]:
preprocessor = make_column_transformer(
    (StandardScaler(), num_col),
    (OneHotEncoder(handle_unknown="ignore", drop="first"), str_col))

NameError: name 'num_col' is not defined

In [113]:
clf = make_pipeline(preprocessor, XGBRegressor(seed=42))

In [114]:
X = dft.drop('target', axis=1)
y = dft['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [115]:
clf.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['baths', 'fireplace', 'sqft',
                                                   'beds', 'stories',
                                                   'private_pool', 'year_built',
                                                   'cooling', 'lot_size',
                                                   'school_rate_mean',
                                                   'school_distance_mean',
                                                   'has_parking', 'garage',
                                                   'parking_spaces']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore')...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [116]:
y_train_pred = clf.predict(X_train)
y_test_pred  = clf.predict(X_test)
print('Train RMSE:', mean_squared_error(y_train, y_train_pred, squared=False))
print('Test RMSE:', mean_squared_error(y_test, y_test_pred, squared=False))

Train RMSE: 295612.07665623137
Test RMSE: 315305.1306865405


In [50]:
print('Train RMSE:', r2_score(y_train, y_train_pred))
print('Test RMSE:', r2_score(y_test, y_test_pred))

Train RMSE: 0.3351674032525429
Test RMSE: 0.2568350511283851


In [38]:
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }
    model = XGBRegressor(**param)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    return mean_squared_error(y_test, y_pred, squared=False)


In [39]:
# Create the study
study = optuna.create_study(direction='minimize', study_name='regression')
study.optimize(objective, n_trials=10)


[I 2023-11-27 18:14:49,492] A new study created in memory with name: regression
[I 2023-11-27 18:14:56,871] Trial 0 finished with value: 398445.9956703492 and parameters: {'max_depth': 1, 'learning_rate': 0.41753849123143616, 'n_estimators': 105, 'min_child_weight': 7, 'gamma': 0.7396079174152242, 'subsample': 0.2671811582443633, 'colsample_bytree': 0.8491262030781767, 'reg_alpha': 0.5143726253932969, 'reg_lambda': 0.461045285970103, 'random_state': 977}. Best is trial 0 with value: 398445.9956703492.
[I 2023-11-27 18:17:33,355] Trial 1 finished with value: 1017494.6196596569 and parameters: {'max_depth': 9, 'learning_rate': 0.6078608834820108, 'n_estimators': 886, 'min_child_weight': 6, 'gamma': 0.3529235169312115, 'subsample': 0.16339745818124207, 'colsample_bytree': 0.43604602363088185, 'reg_alpha': 0.02368542665831514, 'reg_lambda': 0.9207731010581247, 'random_state': 126}. Best is trial 0 with value: 398445.9956703492.
[I 2023-11-27 18:18:50,403] Trial 2 finished with value: 31160

In [40]:
# Print the best parameters
print('Best parameters', study.best_params)


Best parameters {'max_depth': 8, 'learning_rate': 0.5857603430263628, 'n_estimators': 712, 'min_child_weight': 10, 'gamma': 0.8608770141069374, 'subsample': 0.8914373690290588, 'colsample_bytree': 0.8022520819057581, 'reg_alpha': 0.8200382160134515, 'reg_lambda': 0.882237576161878, 'random_state': 178}


In [41]:
# Print the best value
print('Best value', study.best_value)


Best value 301441.8453243442


In [42]:
model = XGBRegressor(**study.best_params)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

print('MSE: ', mean_squared_error(y_test, y_pred))
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred)))


MSE:  90867186112.54582
RMSE:  301441.8453243442


In [ ]:
filename = "data/xgbregressor.pickle"
pickle.dump(model, open(filename, "wb"))


In [44]:
# print('Train RMSE:', r2_score(y_train, y_train_pred))
print('Test RMSE:', r2_score(y_test, y_pred))

Test RMSE: 0.7258033230380283


In [ ]:
# def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
#     """_summary_

#     the Root Mean Squared Log Error (RMSLE) metric
#     """
#     return np.sqrt(mean_squared_log_error(y_true,y_pred))

In [ ]:
# # target function

# def objective(trial, X, y):
#     param_grid = {
#         "n_estimators": trial.suggest_categorical("n_estimators", [1000]),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
#         "num_leaves": trial.suggest_int("num_leaves", 20, 300, step=50),
#         "max_depth": trial.suggest_int("max_depth", 3, 10, step=2),
#         "min_child_samples":trial.suggest_int("min_child_samples", 5, 200, step=50)
#     }
    
#     cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    
#     cv_predicts = np.empty(5)
#     for idx, (train_idx, test_idx) in enumerate(cv.split(X,y)):
#         X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#         y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
#         pruning_callback = optuna.integration.LightGBMPruningCallback(
#             trial, "l2")
#         model = XGBRegressor(**param_grid)
#         model.fit(X_train_scaled,
#                   y_train,
#                   eval_set = [(X_test_scaled, y_test)],
#                   eval_metric="rmse",
#                   callbacks=[pruning_callback]
#                 #   verbose = -1
#                   # early_stopping_rounds = 100
#                   )
#     preds = model.predict(X_test)
#     cv_predicts[idx] = rmsle(y_test, preds)
    
#     return np.mean(cv_predicts)      
        
        

In [ ]:
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0.01, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 1.0),
        'random_state': trial.suggest_int('random_state', 1, 1000)
    }

In [21]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 50, 500, 50)
  max_depth = trial.suggest_int('max_depth', 1, 30, 1)
  min_child_weight = trial.suggest_int('min_child_weight', 1, 10, 1)

  # создаем модель
  model = XGBRegressor(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_child_weight=min_child_weight,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train_scaled, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train_scaled))

  return score

In [22]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="XGboostry", direction="minimize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=10)

[I 2023-11-27 22:21:34,989] A new study created in memory with name: XGboostry
[W 2023-11-27 22:23:35,017] Trial 0 failed with parameters: {'n_estimators': 450, 'max_depth': 10, 'min_child_weight': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\IDE\Skillfactory\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\John Lee\AppData\Local\Temp\ipykernel_24444\743921878.py", line 13, in optuna_rf
    model.fit(X_train_scaled, y_train)
  File "c:\IDE\Skillfactory\.venv\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\IDE\Skillfactory\.venv\lib\site-packages\xgboost\sklearn.py", line 1025, in fit
    self._Booster = train(
  File "c:\IDE\Skillfactory\.venv\lib\site-packages\xgboost\core.py", line 620, in inner_f
    return func(**kwargs)
  File "c:\IDE\Skillfactory\.venv\lib\site-packages\xgboost\training.py", line 185, 

KeyboardInterrupt: 